In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import face_recognition
import json
from tqdm import tqdm

In [2]:
tqdm.pandas()

In [3]:
with open('../data/fer2013.csv', 'r') as f:
    data = pd.read_csv(f)
data

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
5,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
6,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
7,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
8,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training
9,2,255 254 255 254 254 179 122 107 95 124 149 150...,Training


In [4]:
with open('../data/fer2013new.csv', 'r') as f:
    labels = pd.read_csv(f)
labels

,Usage,Image name,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,NF
0,Training,fer0000000.png,4,0,0,1,3,2,0,0,0,0
1,Training,fer0000001.png,6,0,1,1,0,0,0,0,2,0
2,Training,fer0000002.png,5,0,0,3,1,0,0,0,1,0
3,Training,fer0000003.png,4,0,0,4,1,0,0,0,1,0
4,Training,fer0000004.png,9,0,0,1,0,0,0,0,0,0
5,Training,fer0000005.png,6,0,0,1,0,0,1,1,1,0
6,Training,fer0000006.png,2,0,0,8,0,0,0,0,0,0
7,Training,fer0000007.png,0,10,0,0,0,0,0,0,0,0
8,Training,fer0000008.png,0,10,0,0,0,0,0,0,0,0
9,Training,fer0000009.png,0,0,6,0,0,0,4,0,0,0


In [5]:
datas = pd.concat([data, labels], axis=1)
datas

,emotion,pixels,Usage,Usage,Image name,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,NF
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training,Training,fer0000000.png,4,0,0,1,3,2,0,0,0,0
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training,Training,fer0000001.png,6,0,1,1,0,0,0,0,2,0
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training,Training,fer0000002.png,5,0,0,3,1,0,0,0,1,0
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training,Training,fer0000003.png,4,0,0,4,1,0,0,0,1,0
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training,Training,fer0000004.png,9,0,0,1,0,0,0,0,0,0
5,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training,Training,fer0000005.png,6,0,0,1,0,0,1,1,1,0
6,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training,Training,fer0000006.png,2,0,0,8,0,0,0,0,0,0
7,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training,Training,fer0000007.png,0,10,0,0,0,0,0,0,0,0
8,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training,Training,fer0000008.png,0,10,0,0,0,0,0,0,0,0
9,2,255 254 255 254 254 179 122 107 95 124 149 150...,Training,Training,fer0000009.png,0,0,6,0,0,0,4,0,0,0


In [6]:
def pixel_format(pixel_str):
    pixels = np.array([pixel for pixel in pixel_str.split(' ')])
    pixels = pixels.astype('uint8')
    pixels = pixels.reshape(48, 48)
    pixels_rgb = cv2.cvtColor(pixels, cv2.COLOR_GRAY2RGB)
    return pixels_rgb

In [7]:
def make_landmarks(pixel_str):
    pixels_rgb = pixel_format(pixel_str)
    face_landmarks_list = face_recognition.face_landmarks(pixels_rgb)
    if face_landmarks_list:
        return json.dumps(face_landmarks_list[0])
    return np.nan

In [9]:
datas['landmarks'] = datas['pixels'].progress_map(make_landmarks)
datas

100%|██████████| 35887/35887 [03:03<00:00, 195.48it/s]


,emotion,pixels,Usage,Usage,Image name,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,NF,landmarks
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training,Training,fer0000000.png,4,0,0,1,3,2,0,0,0,0,"{""chin"": [[-1, 15], [-2, 22], [-2, 28], [-1, 3..."
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training,Training,fer0000001.png,6,0,1,1,0,0,0,0,2,0,"{""chin"": [[5, 25], [6, 29], [7, 33], [9, 37], ..."
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training,Training,fer0000002.png,5,0,0,3,1,0,0,0,1,0,"{""chin"": [[7, 13], [5, 17], [4, 22], [3, 27], ..."
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training,Training,fer0000003.png,4,0,0,4,1,0,0,0,1,0,NaN
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training,Training,fer0000004.png,9,0,0,1,0,0,0,0,0,0,"{""chin"": [[2, 20], [3, 25], [4, 30], [5, 35], ..."
5,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training,Training,fer0000005.png,6,0,0,1,0,0,1,1,1,0,"{""chin"": [[3, 23], [4, 28], [4, 33], [5, 38], ..."
6,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training,Training,fer0000006.png,2,0,0,8,0,0,0,0,0,0,NaN
7,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training,Training,fer0000007.png,0,10,0,0,0,0,0,0,0,0,"{""chin"": [[10, 22], [10, 25], [10, 28], [11, 3..."
8,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training,Training,fer0000008.png,0,10,0,0,0,0,0,0,0,0,"{""chin"": [[3, 20], [4, 25], [6, 29], [8, 34], ..."
9,2,255 254 255 254 254 179 122 107 95 124 149 150...,Training,Training,fer0000009.png,0,0,6,0,0,0,4,0,0,0,"{""chin"": [[4, 20], [4, 26], [5, 31], [7, 37], ..."


In [ ]:
# data_neutral = datas[datas['neutral'] >= 6]
# data_happiness = datas[datas['happiness'] >= 6]
# data_anger = datas[datas['anger'] >= 5]
# print(len(data_neutral))
# print(len(data_happiness))
# print(len(data_anger))

In [10]:
datas_output = datas[['landmarks', 'pixels', 'neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt', 'unknown']]
datas_output

,landmarks,pixels,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown
0,"{""chin"": [[-1, 15], [-2, 22], [-2, 28], [-1, 3...",70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,4,0,0,1,3,2,0,0,0
1,"{""chin"": [[5, 25], [6, 29], [7, 33], [9, 37], ...",151 150 147 155 148 133 111 140 170 174 182 15...,6,0,1,1,0,0,0,0,2
2,"{""chin"": [[7, 13], [5, 17], [4, 22], [3, 27], ...",231 212 156 164 174 138 161 173 182 200 106 38...,5,0,0,3,1,0,0,0,1
3,NaN,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,4,0,0,4,1,0,0,0,1
4,"{""chin"": [[2, 20], [3, 25], [4, 30], [5, 35], ...",4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,9,0,0,1,0,0,0,0,0
5,"{""chin"": [[3, 23], [4, 28], [4, 33], [5, 38], ...",55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,6,0,0,1,0,0,1,1,1
6,NaN,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,2,0,0,8,0,0,0,0,0
7,"{""chin"": [[10, 22], [10, 25], [10, 28], [11, 3...",77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,0,10,0,0,0,0,0,0,0
8,"{""chin"": [[3, 20], [4, 25], [6, 29], [8, 34], ...",85 84 90 121 101 102 133 153 153 169 177 189 1...,0,10,0,0,0,0,0,0,0
9,"{""chin"": [[4, 20], [4, 26], [5, 31], [7, 37], ...",255 254 255 254 254 179 122 107 95 124 149 150...,0,0,6,0,0,0,4,0,0


In [11]:
with open('../data/fer_with_landmarks.csv', 'w') as f:
    datas_output.to_csv(f)

In [14]:
for index, row in tqdm(datas_output.iterrows()):
    plt.imsave('../data/dataset_images/fer%07d.png' % index, pixel_format(row['pixels']))

35887it [02:21, 253.07it/s]
